In [ ]:
import requests
import json
from io import StringIO
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import folium
import branca
import branca.colormap as cm
from branca.colormap import linear
import geopandas
from folium.features import GeoJsonTooltip

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
data = requests.get("https://raw.githubusercontent.com/allisol1/UMBC-CMSC631-Team6/refs/heads/main/ACS_5YR_Housing_Estimate_Data_by_County_2352642343660635057.csv")
df_h_full = pd.read_csv(StringIO(data.text))
data = requests.get("https://raw.githubusercontent.com/allisol1/UMBC-CMSC636-Team6/refs/heads/main/DD_ACS_5-Year_Housing_Estimate_Data_by_County.csv")
df_keys = pd.read_csv(StringIO(data.text))
data = requests.get("https://raw.githubusercontent.com/allisol1/UMBC-CMSC636-Team6/refs/heads/main/county_adjacency2024.txt")
df_adj = pd.read_csv(StringIO(data.text), sep='|')

In [ ]:
state_geojson_full = geopandas.read_file("ACS_5YR_Housing_Estimate_Data_by_State_1435040831713560726.geojson")
county_geojson_full = geopandas.read_file("ACS_5YR_Housing_Estimate_Data_by_County_318069920734049109.geojson")

In [ ]:
print(county_geojson_full.columns.values)

In [ ]:
state_geojson = state_geojson_full[['GEOID', 'STUSAB', 'NAME','B25002EST1', 'B25002EST2', 'B25058EST1', 'B25032EST13', 'geometry']].copy()
county_geojson = county_geojson_full[['GEOID', 'STUSAB', 'STATE_NAME', 'NAME','B25002EST1', 'B25002EST2', 'B25058EST1', 'B25032EST13', 'geometry']].copy()

In [ ]:
with requests.get("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(StringIO(response.text))
counties['features'][0]

In [ ]:
fig = px.choropleth(county_geojson, geojson=counties, locations='GEOID', color='B25058EST1',
                           color_continuous_scale="BuPu",
                           range_color=(0, county_geojson['B25058EST1'].max()),
                           scope="usa",
                           labels={'B25058EST1':'Median Rent'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
county_geojson.head(5)

In [ ]:
cm.linear

In [ ]:
# ideal_rent = int(input("Enter ideal rent price: "))

# base map, start at center of US
m = folium.Map(location=[48, -102], zoom_start=4)

# remove null values so that it can work with colormap
county_geojson.B25058EST1 = county_geojson.B25058EST1.fillna(np.nan)
county_geojson = county_geojson.dropna(subset=['B25058EST1'])

# # calculate difference of median rent and ideal rent
# county_geojson['IDEAL_RENT_DIFF'] = (county_geojson['B25058EST1'] - ideal_rent)

# # sets color to difference from ideal rent price, may need to change indexes (>400, 400-499, 500-599, etc)
# diverging_colormap = linear.PiYG_05.scale(
#     county_geojson.IDEAL_RENT_DIFF.min(), county_geojson.IDEAL_RENT_DIFF.max()
# )

# sets color to rent price, need to change indexes (>400, 400-499, 500-599, etc)
sequential_colormap = linear.BuPu_09.scale(
    county_geojson.B25058EST1.min(), county_geojson.B25058EST1.max()
)

# # display legend for ideal rent difference
# legend = cm.StepColormap(
#     colors=diverging_colormap.colors,
#     index=diverging_colormap.index,
#     vmin=county_geojson.IDEAL_RENT_DIFF.min(),
#     vmax=county_geojson.IDEAL_RENT_DIFF.max(),
#     caption="Difference from Ideal Rent Price"
# )
# m.add_child(legend)

# display legend for median rent price
legend = cm.StepColormap(
    colors=sequential_colormap.colors,
    index=sequential_colormap.index,
    vmin=county_geojson.B25058EST1.min(),
    vmax=county_geojson.B25058EST1.max(),
    caption="Median Rent Price"
)
m.add_child(legend)

county_geojson.head(5)

# # create choropleth for ideal rent difference
# folium.GeoJson(
#     county_geojson,
#     name = "IDEAL_RENT_DIFF",
#     style_function=lambda feature: {
#         "fillColor": diverging_colormap(feature["properties"]["IDEAL_RENT_DIFF"]),
#         "color": "black",
#         "weight": 1,
#         "fillOpacity": 1
#     },
#     tooltip=GeoJsonTooltip(
#        fields=["NAME", "B25058EST1", "IDEAL_RENT_DIFF"],
#        aliases=["County:", "Median Rent Price:", "Ideal Rent Difference:"],
#    )
# ).add_to(m)

# create choropleth for median rent price
folium.GeoJson(
    county_geojson,
    name = "B25058EST1",
    style_function=lambda feature: {
        "fillColor": sequential_colormap(feature["properties"]["B25058EST1"]),
        "opacity": 0.1,
        "color": "black",
        "weight": 1,
        "fillOpacity": 1
    },
    tooltip=GeoJsonTooltip(
       fields=["NAME", "B25058EST1"],
       aliases=["County:", "Median Rent Price:"],
   )
).add_to(m)

# display map
m.save('map.html')
m

In [ ]:
ideal_rent = int(input("Enter ideal rent price: "))

# base map, start at center of US
m = folium.Map(location=[48, -102], zoom_start=4)

# calculate difference of median rent and ideal rent
state_geojson['IDEAL_RENT_DIFF'] = (state_geojson['B25058EST1'] - ideal_rent)

# display new data
state_geojson.head(10)

# sets color to difference from ideal rent price, may need to change indexes (>400, 400-499, 500-599, etc)
diverging_colormap = linear.PiYG_05.scale(
    state_geojson.IDEAL_RENT_DIFF.min(), state_geojson.IDEAL_RENT_DIFF.max()
)

# sets color to rent price, need to change indexes (>400, 400-499, 500-599, etc)
sequential_colormap = linear.BuPu_09.scale(
    state_geojson.B25058EST1.min(), state_geojson.B25058EST1.max()
)

# display legend for ideal rent difference
legend = cm.StepColormap(
    colors=diverging_colormap.colors,
    index=diverging_colormap.index,
    vmin=state_geojson.IDEAL_RENT_DIFF.min(),
    vmax=state_geojson.IDEAL_RENT_DIFF.max(),
    caption="Difference from Ideal Rent Price"
)
m.add_child(legend)

# display legend for median rent price
legend = cm.StepColormap(
    colors=sequential_colormap.colors,
    index=sequential_colormap.index,
    vmin=state_geojson.B25058EST1.min(),
    vmax=state_geojson.B25058EST1.max(),
    caption="Median Rent Price"
)
m.add_child(legend)

# create choropleth ideal rent difference
folium.GeoJson(
    state_geojson,
    name = "Difference from Ideal Rent Price",
    style_function=lambda feature: {
        "fillColor": diverging_colormap(feature["properties"]["IDEAL_RENT_DIFF"]),
        "color": "black",
        "weight": 1,
        "fillOpacity": 1
    },
    tooltip=GeoJsonTooltip(
       fields=["NAME", "B25058EST1", "IDEAL_RENT_DIFF"],
       aliases=["State:", "Median Rent Price:", "Difference from Idea Rent Price:"],
   )
).add_to(m)

# create choropleth for median rent price
folium.GeoJson(
    state_geojson,
    name = "Median Rent Price",
    style_function=lambda feature: {
        "fillColor": sequential_colormap(feature["properties"]["B25058EST1"]),
        "color": "black",
        "weight": 1,
        "fillOpacity": 1
    },
    tooltip=GeoJsonTooltip(
       fields=["NAME", "B25058EST1"],
       aliases=["State:", "Median Rent:"],
   )
).add_to(m)

m.add_child(folium.LayerControl())

# display map
m

In [ ]:
df_keys

In [ ]:
df_h = county_geojson.loc[county_geojson['STATE_NAME'] == "Maryland",  ['GEOID', 'NAME', 'STUSAB','B25002EST1', 'B25002EST2', 'B25058EST1', 'B25032EST13']]
df_h
df_h['PCT_RENTER'] = (df_h['B25032EST13'] / df_h['B25002EST2']) * 100
df_h